In [1]:
import sys; sys.path.insert(0,'..')
from ppanlp import *

In [2]:
# get_ppa_metadata_d()

In [3]:
iterr=iter_pages_hathi()

In [4]:
# d_hathi=next(iterr)

In [5]:
# print(list(d_hathi.keys()))

In [6]:
# d_hathi['pageid']

In [7]:
iterr_ecco=iter_pages_ecco()

In [8]:
# d_ecco=next(iterr_ecco)

In [9]:
# d_ecco['pageid']

In [10]:
# print(list(d_ecco.keys()))

In [11]:
# d_ecco

In [12]:
# common_keys = set(d_hathi.keys()) & set(d_ecco.keys())
# print(common_keys)

In [13]:
iterr_all = iterate_over_all_ppa_pages()

In [14]:
# next(iterr_all)

In [15]:
def save_ppa_pages():
    iterr_all = iterate_over_all_ppa_pages()
    with get_ppa_page_cache() as cache:
        for d in tqdm(iterr_all):
            pageid=d['pageid']
            txt=d['text']
            source_id=d['source_id']
            cache[f'{source_id}_{pageid}']=txt

In [16]:
# save_ppa_pages()

In [17]:
# keys=list(get_ppa_page_cache().keys())
# keys.sort()

In [18]:
# with open(PATH_PPA_PAGES_CACHE_KEYS,'w') as of: of.write('\n'.join(keys))
with open(PATH_PPA_PAGES_CACHE_KEYS,'r') as f: keys=f.read().split('\n')

In [19]:
def save_ppa_text_corpus(page_sep='\n\n\n', force=True):
    os.makedirs(PATH_TEXT_CORPUS_TEXTS, exist_ok=True)
    metad=get_ppa_metadata_d()
    source_id=None
    pages_text = []
    meta_ld=[]
    
    def save_current(source_id,pages_text):
        if not source_id in metad:
            print('!! ERROR:',source_id,'not found in metadata')
            return {}
        pagerange=metad[source_id]['pages_orig']
        filename=f'{source_id}_{pagerange}.txt' if pagerange else f'{source_id}.txt'
        filename=filename.replace('/','[').replace('\\',']').replace(' ','')
        ofn=os.path.join(PATH_TEXT_CORPUS_TEXTS, filename)
        with open(ofn,'w') as of:
            of.write(page_sep.join(pages_text))
        return {'filename':filename, **metad[source_id]}
    
    with get_ppa_page_cache() as cache:
        for sourcepage in tqdm(keys):
            new_source_id,new_pageid = sourcepage.split('_',1) if '_' in sourcepage else (sourcepage,'')
            if source_id is None: source_id=new_source_id
            if new_source_id!=source_id:
                meta_d=save_current(source_id,pages_text)
                if meta_d: meta_ld.append(meta_d)
                source_id = new_source_id
                pages_text = []
            
            txt=cache[sourcepage]
            pages_text.append(txt.strip())

        if pages_text and source_id:
            meta_d=save_current(source_id,pages_text)
            if meta_d: meta_ld.append(meta_d)

    meta_df=pd.DataFrame(meta_ld).set_index('filename')
    meta_df.to_csv(PATH_TEXT_CORPUS_METADATA)
    return meta_df

In [20]:
#### save_ppa_text_corpus()

 48%|███████████████▏                | 832570/1752540 [03:56<04:29, 3415.57it/s]

!! ERROR: njp.32101065672345 not found in metadata


 70%|█████████████████████▌         | 1218054/1752540 [05:46<02:27, 3629.96it/s]

!! ERROR: nyp.33433082273297 not found in metadata


 77%|███████████████████████▉       | 1354581/1752540 [06:24<01:46, 3747.24it/s]

...

!! ERROR: pst.000012612925 not found in metadata


 78%|████████████████████████▏      | 1369694/1752540 [06:28<01:44, 3654.79it/s]

!! ERROR: pst.000029315130 not found in metadata


100%|███████████████████████████████| 1752540/1752540 [08:17<00:00, 3525.19it/s]


,id,record_id,title,subtitle,sort_title,author,item_type,book_journal,pub_date,pub_place,publisher,enumcron,collections,cluster,public_notes,notes,pages_orig,pages_digital,page_count,status,source,added,updated,source_url
filename,,,,,,,,,,,,,,,,,,,,,,,,
CB0126124028.txt,6508,T225266,THE INTRODUCTION.,,INTRODUCTION.,,Excerpt,"A General dictionary of the English language, ...",1768.0,London,"J. and R. Fuller, in Ave-Maria-Lane; and sold ...",,Dictionaries; Linguistic,,,Undecided,,3-4,2.0,Public,Gale,2022-11-03 15:43:03,2022-11-03 15:43:03,https://link.gale.com/apps/doc/CB0126124028/EC...
CB0127060264_iii-xxi.txt,6491,W12782,PREFACE.,,PREFACE.,"Watts, Isaac",Excerpt,"Horae lyricae. Poems, chiefly of the lyric kin...",1798.0,"Printed at Windham, (Connecticut)",by John Byrne,,Literary,wattsprefacehl,,Watts is author,iii-xxi,3-21,19.0,Public,Gale,2022-11-03 15:42:41,2023-08-25 15:54:16,https://link.gale.com/apps/doc/CB0127060264/EC...
CB0127251015_iii-xxiv.txt,6431,T166416,THE PREFACE.,,PREFACE.,"Watts, Isaac",Excerpt,"Horæ lyricæ. Poems, chiefly of the lyric kind:...",1796.0,Glasgow,"James & Andrew Duncan, Booksellers",,Literary,wattsprefacehl,,,iii-xxiv,3-28,26.0,Public,Gale,2022-11-03 15:40:16,2023-08-25 15:52:33,https://link.gale.com/apps/doc/CB0127251015/EC...
CB0127376362.txt,5831,T169662,"The royal English grammar,",Containing what is necessary to the knowledge ...,"royal English grammar, Containing what is nece...","Greenwood, James, d. 1737",Full work,,1761.0,Corke,"Eugene Swiney, near the Exchange",,Linguistic,greenwoodroyal,,6th ed 1761,,,156.0,Public,Gale,2021-08-05 10:31:11,2023-03-02 13:50:01,https://link.gale.com/apps/doc/CB0127376362/EC...
CB0128108681_179-218.txt,6504,N25296,Conjectures on Original Composition. In a Lett...,,Conjectures on Original Composition. In a Lett...,"Young, Edward",Excerpt,"The works of Edward Young, LL.D. in three volu...",1796.0,London,James Mundell and Company,,Literary,,,,179-218,183-222,40.0,Public,Gale,2022-11-03 15:42:56,2022-11-03 15:42:56,https://link.gale.com/apps/doc/CB0128108681/EC...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yale.39002004932761.txt,5004,009833752,"Critical reflections on poetry, painting and m...",With an inquiry into the rise and progress of ...,"Critical reflections on poetry, painting and m...","Dubos, abbé (Jean-Baptiste), 1670-1742",Full work,,1748.0,London,John Nourse,,Literary,duboscritical3,,,,,310.0,Public,HathiTrust,2018-01-23 15:18:45,2023-03-02 13:49:56,https://hdl.handle.net/2027/yale.39002004932761
yale.39002005422606.txt,5003,009833752,"Critical reflections on poetry, painting and m...",With an inquiry into the rise and progress of ...,"Critical reflections on poetry, painting and m...","Dubos, abbé (Jean-Baptiste), 1670-1742",Full work,,1748.0,London,John Nourse,,Literary,duboscritical2,,,,,436.0,Public,HathiTrust,2018-01-23 15:18:44,2023-03-02 13:49:56,https://hdl.handle.net/2027/yale.39002005422606
yale.39002005422614.txt,5002,009833752,"Critical reflections on poetry, painting and m...",With an inquiry into the rise and progress of ...,"Critical reflections on poetry, painting and m...","Dubos, abbé (Jean-Baptiste), 1670-1742",Full work,,1748.0,London,John Nourse,,Literary,duboscritical1,,,,,438.0,Public,HathiTrust,2018-01-23 15:18:44,2023-03-02 13:49:56,https://hdl.handle.net/2027/yale.39002005422614
